In [10]:
from transformers import BertModel, BertTokenizer, RobertaTokenizer,AutoTokenizer, AutoModelForMaskedLM,AutoModel
import torch

model_name = 'DeepChem/ChemBERTa-77M-MLM'

tokenizer = AutoTokenizer.from_pretrained(model_name)
chembert_model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

def get_bert_embeddings(smiles_strings):
    encoded_input = tokenizer(smiles_strings, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = chembert_model(**encoded_input)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Using the [CLS] token embedding from last hidden state
    return embeddings


Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import pandas as pd
train_data = pd.read_csv('/home/parsa/smiles_classification/training_w_features.csv').sample(frac=1)
val_data = pd.read_csv('/home/parsa/smiles_classification/validation_w_features.csv')#.rename({'RESULTS':'RESULT'},axis=1) #pd.read_csv('/home/parsa/smiles_classification/data_validation.csv')


In [3]:
train_data.groupby('Results').count()

,SMILES,Molecular Weight,LogP,Number of Atoms,Number of Bonds,Number of Rings,Rotatable Bonds Count,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Number of Stereocenters,Topological Polar Surface Area (TPSA)
Results,,,,,,,,,,,
0,203,203,203,203,203,203,203,203,203,203,203
1,203,203,203,203,203,203,203,203,203,203,203


In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


def process_X_data(smiles, features):
    scaler = StandardScaler()

    embeddings = get_bert_embeddings(smiles).numpy()
    combined_features = np.concatenate((embeddings, features), axis=1)
    return scaler.fit_transform(combined_features)

In [5]:
feature_columns = ['Molecular Weight', 'LogP', 'Number of Atoms',
       'Number of Bonds', 'Number of Rings', 'Rotatable Bonds Count',
       'Hydrogen Bond Donors', 'Hydrogen Bond Acceptors',
       'Number of Stereocenters', 'Topological Polar Surface Area (TPSA)'] # Add all your feature column names


train_x = process_X_data(train_data.SMILES.tolist(), train_data[feature_columns].values )

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB  # Gaussian for continuous features, Multinomial for discrete
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from functools import partial

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


# Define a pipeline if scaling is needed (e.g., for SVM or KNN)
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

# Define parameter grids for different classifiers
param_grid = [
    {'scaler': [StandardScaler(), MinMaxScaler()],
     'classifier': [SVC()],
     'classifier__C': [0.1, 1, 10],
     'classifier__kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
    {'scaler': [StandardScaler(), MinMaxScaler()],
     'classifier': [RandomForestClassifier()],
     'classifier__n_estimators': [10, 50, 100],
     'classifier__max_features': ['sqrt', 'log2']},
    {'scaler': [StandardScaler(), MinMaxScaler()],
     'classifier': [GradientBoostingClassifier()],
     'classifier__n_estimators': [50, 100, 150],
     'classifier__learning_rate': [0.01, 0.1, 0.2]},
    {'scaler': [StandardScaler(), MinMaxScaler()],
     'classifier': [LogisticRegression(max_iter=1000)],
     'classifier__C': [0.1, 1, 10]},
    {'scaler': [StandardScaler(), MinMaxScaler()],
     'classifier': [KNeighborsClassifier()],
     'classifier__n_neighbors': [3, 5, 7, 9, 11, 13]},
    {'scaler': [StandardScaler(), MinMaxScaler()],
     'classifier': [DecisionTreeClassifier()],
     'classifier__criterion': ['gini', 'entropy'],
     'classifier__max_depth': [None, 10, 20],
     'classifier__min_samples_split': [2, 10]},
    {'scaler': [StandardScaler()],
     'classifier': [GaussianNB()]},
    {'scaler': [MinMaxScaler()],  # Ensure non-negative input for MultinomialNB
     'classifier': [MultinomialNB()],
     'classifier__alpha': [0.1, 1.0, 10.0]},
]
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0),
    'precision': make_scorer(precision_score, zero_division=0)
}
# Create a GridSearchCV object
grid_search = GridSearchCV(
    pipe,
    param_grid,
    scoring=scoring,
    refit='precision',  # Choose one metric to use for refitting the best model
    cv=10,
    return_train_score=True,
    n_jobs=-1
)

# Assuming X_train, y_train are your data prepared earlier
grid_search.fit(train_x,  train_data.Results)

# Best model after grid search
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)


Best parameters: {'classifier': GradientBoostingClassifier(), 'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100, 'scaler': MinMaxScaler()}
Best cross-validated score: 0.7302871283306066


In [7]:
results = grid_search.cv_results_

import pandas as pd

# Convert results to a DataFrame for easier handling and visualization
df_results = pd.DataFrame(results)
# selected_columns = [col for col in df_results.columns if 'param_' in col or 'test_accuracy' in col or 'test_precision' in col]
# df_results = df_results[selected_columns]
display(df_results[['param_classifier','mean_test_precision',	'mean_test_accuracy','mean_test_recall',	'mean_test_f1',	'rank_test_precision']].sort_values('rank_test_precision').drop_duplicates('param_classifier'))


,param_classifier,mean_test_precision,mean_test_accuracy,mean_test_recall,mean_test_f1,rank_test_precision
39,GradientBoostingClassifier(),0.730287,0.726463,0.728810,0.725804,1
26,RandomForestClassifier(),0.691450,0.672622,0.635714,0.660238,7
14,SVC(),0.676809,0.689451,0.743095,0.705566,10
84,DecisionTreeClassifier(),0.664654,0.662195,0.665476,0.663622,16
55,LogisticRegression(max_iter=1000),0.664570,0.669695,0.699286,0.678140,17
97,MultinomialNB(),0.617277,0.615549,0.605476,0.606917,70
61,KNeighborsClassifier(),0.597963,0.598232,0.620000,0.607355,74
96,GaussianNB(),0.595014,0.603415,0.639762,0.612426,75


In [11]:
import pandas as pd

# Extract best models for each classifier type
results = pd.DataFrame(grid_search.cv_results_)
best_models = {}
val_data = pd.read_csv('/home/parsa/smiles_classification/validation_w_features.csv')#.rename({'RESULTS':'RESULT'},axis=1) #pd.read_csv('/home/parsa/smiles_classification/data_validation.csv')
val_x = process_X_data(val_data.SMILES.tolist(), val_data[feature_columns].values )

for classifier_name in ["SVC", "RandomForestClassifier", "GradientBoostingClassifier", 
                        "LogisticRegression", "KNeighborsClassifier", 
                        "DecisionTreeClassifier", "GaussianNB", "MultinomialNB"]:
    best_result = results[results['param_classifier'].apply(lambda x: x.__class__.__name__) == classifier_name].iloc[0]
    best_params = best_result['params']
    
    # Recreate the pipeline with the best parameters
    best_pipeline = Pipeline([
        ('scaler', best_params['scaler']),
        ('classifier', best_params['classifier'])
    ])
    best_pipeline.set_params(**{k: v for k, v in best_params.items() if k not in ['scaler', 'classifier']})
    
    # Store the model for validation
    best_models[classifier_name] = best_pipeline

# Validation step
validation_results = []
for classifier_name, model in best_models.items():
    model.fit(train_x, train_data.Results)  # Refit on the training data
    y_pred = model.predict(val_x)
    # Calculate metrics
    accuracy = accuracy_score(val_data.RESULT, y_pred)
    precision = precision_score(val_data.RESULT, y_pred, zero_division=0)
    recall = recall_score(val_data.RESULT, y_pred, zero_division=0)
    f1 = f1_score(val_data.RESULT, y_pred, zero_division=0)
    
    # Append results to the list
    validation_results.append({
        "Model": classifier_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

# Convert the results to a dataframe
validation_results_df = pd.DataFrame(validation_results).sort_values('Accuracy', ascending=False)

# Display the dataframe
display(validation_results_df)


,Model,Accuracy,Precision,Recall,F1 Score
2,GradientBoostingClassifier,0.72,0.703704,0.76,0.730769
3,LogisticRegression,0.62,0.625000,0.60,0.612245
4,KNeighborsClassifier,0.62,0.607143,0.68,0.641509
5,DecisionTreeClassifier,0.62,0.600000,0.72,0.654545
6,GaussianNB,0.60,0.592593,0.64,0.615385
0,SVC,0.58,0.583333,0.56,0.571429
1,RandomForestClassifier,0.54,0.550000,0.44,0.488889
7,MultinomialNB,0.54,0.538462,0.56,0.549020
